In [ ]:
import os
import cv2
import numpy as np
from skimage.color import rgb2gray
from scipy import ndimage

In [ ]:
def fa_processing(img_path: str, size_threshold: int, scale: float, output_img_path: str):
    '''
    Returns numpy arrays for the binary and segmented FA images from the raw FA images.

        Parameters:
                img_path: A string of the path to the image containing raw FA
                size_threshold: An integer value for size threshold; any FAs with an area less than this value are dropped
                scale: A float value for image scale (microns/pixel)
                output_img_path: A string of the path to the output binary FA image

        Returns:
                binary_img: A numpy array of the binary FA image
                seg_img: A numpy array of the segmented FA image
                output_img: Writes the binary FA image to user defined path
    '''
    # Read image and extract FA channel into a new empty image
    img = cv2.imread(img_path)
    res = np.zeros((256, 256, 3))
    res[:,:,1] = img[:,:,1]
    # Apply a top hat filter with 3x3 rectangular kernel and place filtered image into a new empty image
    filterSize =(3, 3)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)
    tophat_img = cv2.morphologyEx(res[:,:,1], cv2.MORPH_TOPHAT, kernel)        
    processed_img = res
    processed_img[:,:,1] = tophat_img
    # Binarize filtered FA image
    ret, bw_img = cv2.threshold(processed_img,10,255,cv2.THRESH_BINARY)
    gray = rgb2gray(bw_img)
    gray = gray * 255
    gray = gray.astype(np.uint8)
    ret, out_l = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    # Find the contours of each FA
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    binary_img = np.zeros((256,256))
    # Iterate through each contour
    for cntr in contours:
        # Use bouding rectangle to find width and height of each FA to calculate area
        x,y,w,h = cv2.boundingRect(cntr)
        w = w*scale
        h = h*scale
        area = w*h
        # Write FAs that have an area equal or greater than the threshold into a new empty image
        if area >= size_threshold:
            cv2.drawContours(binary_img, [cntr], 0, (255, 0, 0), -1)
    # Apply laplacian operator to binary FA image to obtain segmented FA image
    kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
    seg_img = ndimage.convolve(binary_img, kernel_laplace, mode='constant')
    seg_img = seg_img * 255
    ret, seg_img = cv2.threshold(seg_img,10,255,cv2.THRESH_BINARY)
    output_img = cv2.imwrite(output_img_path, binary_img)
    return binary_img, seg_img, output_img

In [ ]:
def membrane_processing(img_path: str, output_img_path: str):
    '''
    Returns numpy array for the segmented membrane image from the raw membrane image.

        Parameters:
                img_path: A string of the path to the image containing raw membrane
                output_img_path: A string of the path to the output segmented membrane image

        Returns:
                seg_img: A numpy array of the segmented membrane image
                output_img: Writes the segmented membrane image to user defined path
    '''
    # Read image and extract membrane channel to binarize
    membrane = cv2.imread(img_path)
    ret, bw_img = cv2.threshold(membrane[:,:,2],10,255,cv2.THRESH_BINARY)
    # Place binary membrane channel into a new empty image
    res = np.zeros((256, 256, 3))
    res[:,:,2] = bw_img
    gray = rgb2gray(res)
    # Apply laplacian operator to binary membrane image to obtain segmented membrane image
    kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
    out_l = ndimage.convolve(gray, kernel_laplace, mode='constant')
    out_l = out_l * 255
    ret, seg_img = cv2.threshold(out_l,10,255,cv2.THRESH_BINARY)
    output_img = cv2.imwrite(output_img_path, seg_img)
    return seg_img, output_img

In [ ]:
membrane_processing('unprocessed_ground_truth.png', 'membrane_segmented2.png')

In [ ]:
fa_processing('unprocessed_ground_truth.png', 1, 0.7, 'FA_truth_binary2.png')